In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import json
import matplotlib.pyplot as plt

mlb = MultiLabelBinarizer()

dir_proj            = "S:/Wissdaten/ZKI-PH4/sprint_mosqlimate/"
dir_data            = os.path.join(dir_proj, 'data')
dir_data_processed  = os.path.join(dir_data, 'processed')

def add_week_features(df):
    """
    adds sin and cos values per week
    """
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df["weekofyear"] = (df['date'].dt.isocalendar().week - 1) / 52
    df["week_sin"]     = np.sin(2 * np.pi * df["weekofyear"])
    df["week_cos"]     = np.cos(2 * np.pi * df["weekofyear"])

    df.drop(columns=['weekofyear'], inplace=True)
    return df

def pipeline_minmax_scaling(train_df, columns):
    """
    takes training data and standardizes the columns specified into minmax
    
    returns the same dataframe with the specified columns standardized. Other columns remain untouched.
    """
    params = {}
    scaled_df = train_df.copy()
    
    for col in columns:
        col_min = train_df[col].min()
        col_max = train_df[col].max()
        params[col] = {'min': col_min, 'max': col_max}
        
        if col_max - col_min == 0:
            scaled_df[col] = 0.0
        else:
            scaled_df[col] = (train_df[col] - col_min) / (col_max - col_min)
    
    return scaled_df, params

def apply_minmax_scaling(val_df, columns, params):
    """
    takes data and standardizes following parameters given. 
    This will be used for the validation dataset after having standardized the training dataset.

    returns scaled data
    """
    scaled_df = val_df.copy()
    
    for col in columns:
        if col not in params:
            raise ValueError(f"Scaling parameters for column '{col}' not found.")
        
        col_min = params[col]['min']
        col_max = params[col]['max']
        
        if col_max - col_min == 0:
            scaled_df[col] = 0.0
        else:
            scaled_df[col] = (val_df[col] - col_min) / (col_max - col_min)
    
    return scaled_df

def load_and_merge_rawdata(aggr_level = 'state'):
    """"
    individually loads each separate rawdata file and then merges it all, based on a given aggregation level. This is either:
    "state", "macroregion" or "region"
    
    """
    aggr_level_map =   {'state'          : 'uf',
                        'macroregion'    : 'macroregional_geocode',
                        'region'         : 'regional_geocode'}

    aggr_level_options = list(aggr_level_map.keys())

    if aggr_level not in aggr_level_options:
        raise ValueError(f'{aggr_level} not valid! options are {aggr_level_options}')

    aggr_level_colname = aggr_level_map[aggr_level]

    df_cases_raw                            = pd.read_csv(os.path.join(dir_data, 'dengue.csv.gz'))
    df_cases_raw['year']                    = pd.to_datetime(df_cases_raw['date']).dt.year

    df_cases_raw                            = df_cases_raw[df_cases_raw['uf']!= 'ES']

    df_cases_aggr                           = df_cases_raw.groupby(['year',aggr_level_colname,'epiweek','date'])['casos'].sum().reset_index()
    df_cases_aggr                           = df_cases_aggr[['date','epiweek',aggr_level_colname,'year','casos']]
    geocodes_harm                           = df_cases_raw[[aggr_level_colname,'geocode']].drop_duplicates()

    df_population_raw                       = pd.read_csv(os.path.join(dir_data, 'datasus_population_2001_2024.csv.gz'))
    df_population_raw                       = pd.merge(df_population_raw, geocodes_harm, on ='geocode')
    df_population_aggr                      = df_population_raw.groupby(['year',aggr_level_colname])['population'].sum().reset_index()   

    # Project 2025 population by adding the same difference to 2024
    population_projection_df = df_population_aggr.copy()
    population_projection_df        = population_projection_df.pivot(index=aggr_level_colname, columns='year', values='population')
    population_projection_df['diff']= population_projection_df[2024] - population_projection_df[2023]
    population_projection_df[2025]  = population_projection_df[2024] + population_projection_df['diff']
    population_projection_df_2025   = population_projection_df[[2025]].reset_index()
    population_projection_df_2025['year'] = 2025
    population_projection_df_2025.rename(columns={2025: 'population'}, inplace=True)
        # Match the original DataFrame format
    population_projection_df_2025 = population_projection_df_2025[['year', aggr_level_colname, 'population']]
        # Append to the original DataFrame
    df_population_aggr_updated = pd.concat([df_population_aggr, population_projection_df_2025], ignore_index=True)
    df_population_aggr_updated = df_population_aggr_updated.sort_values(by=['year', aggr_level_colname]).reset_index(drop=True)

    df                                      = pd.merge(df_cases_aggr, df_population_aggr_updated, on = ['year',aggr_level_colname])
    df['incidence']                         = df['casos'] / df['population'] * 10_000
    df['date']                              = pd.to_datetime(df['date'])
    df.rename(columns={'casos': 'cases'}, inplace=True)

    # Climate data
    df_climate_raw                          = pd.read_csv(os.path.join(dir_data, 'climate.csv.gz'))
    df_climate_raw['date']                  = pd.to_datetime(df_climate_raw['date'])
    df_climate_raw                          = pd.merge(df_climate_raw, geocodes_harm, on ='geocode')

    # Area data
    area_data_raw                           = pd.read_csv(os.path.join(dir_proj, 'land_coverage_cleaned.csv'))
    area_data_raw                           = area_data_raw[['geocode', 'AREA_HECTARES']].drop_duplicates()
    climate_area_raw                        = pd.merge(df_climate_raw, area_data_raw, on ='geocode')

    exclude_cols                            = ['date', 'epiweek', 'geocode', aggr_level_colname, 'AREA_HECTARES']
    climate_cols                            = [col for col in climate_area_raw.select_dtypes(include='number').columns if col not in exclude_cols]

    def weighted_mean(group, value_cols, weight_col):
        weights = group[weight_col]
        # For each climate column, calculate weighted average using municipality's area
        return pd.Series({
            col: (group[col] * weights).sum() / weights.sum()
            for col in value_cols
        })

    # Group by 'date' and aggregation level and apply weighted mean towards area
    df_climate_aggr         = climate_area_raw.groupby(['date', aggr_level_colname]).apply(weighted_mean, climate_cols, 'AREA_HECTARES').reset_index()
    df_climate_aggr['date'] = pd.to_datetime(df_climate_aggr['date'])
    df                      = pd.merge(df, df_climate_aggr, on = ['date',aggr_level_colname])

    # ocean climate data
    df_ocean_climate                        = pd.read_csv(os.path.join(dir_data, 'ocean_climate_oscillations.csv.gz'))
    df_ocean_climate['date']                = pd.to_datetime(df_ocean_climate['date'])

    # dates from ocean climate data didn't entirely match with the other datasets. 
    df = pd.merge_asof(
        df.sort_values('date'),
        df_ocean_climate.sort_values('date'),
        on        = 'date',
        direction ='nearest',
        tolerance = pd.Timedelta('7 days') 
    )
    # environment data
    df_enviroment_raw           = pd.read_csv(os.path.join(dir_data,'environmental_additions.csv'))

    rename_environment_cols = {
    'Coastal_Water_Body-sum_normalized'        :   'coastal',
    'Continental_Water_Body-sum_normalized'    :   'water_body',
    'Depressions-sum_normalized'               :   'depressions',
    'Levels-sum_normalized'                    :   'levels',
    'Mountains-sum_normalized'                 :   'mountains',
    'Oceanic_Island-sum_normalized'            :   'oceanic_island',
    'Plains-sum_normalized'                    :   'plains',
    'Plates-sum_normalized'                    :   'plates',
    'Plateus-sum_normalized'                   :   'plateus',
    'Trays-sum_normalized'                     :   'trays',
    'Railways-sum_normalized'                  :   'railways',
    'Amazon_Forest-sum_normalized'             :   'amazon',
    'Atlantic_Forest-sum_normalized'           :   'atlantic_forest',
    'Caatinga-sum_normalized'                  :   'caatinga',
    'Cerrado-sum_normalized'                   :   'cerrado',
    'Pampa-sum_normalized'                     :   'pampa',
    'Pantanal-sum_normalized'                  :   'pantanal',
    'Altitude-mean'                            :   'altitude_mean',
    'Altitude-sd'                              :   'altitude_sd',
    'Altitude-min'                             :   'altitude_min',
    'Altitude-max'                             :   'altitude_max'
    }

    df_enviroment_raw           = df_enviroment_raw.rename(columns = rename_environment_cols)
    df_enviroment_raw           = pd.merge(df_enviroment_raw, area_data_raw, on = ['geocode'])
    df_enviroment_raw           = pd.merge(df_enviroment_raw, geocodes_harm, on = ['geocode'])

    exclude_cols                = ['geocode', aggr_level_colname, 'AREA_HECTARES']
    environment_cols            = [col for col in df_enviroment_raw.select_dtypes(include='number').columns if col not in exclude_cols]


    def weighted_mean(group, value_cols, weight_col):
        weights = group[weight_col]
        # For each climate column, calculate weighted average
        return pd.Series({
            col: (group[col] * weights).sum() / weights.sum()
            for col in value_cols
        })

    # Group by 'date' and 'uf' and apply weighted mean weighted by municipality's area
    df_environment_aggr = df_enviroment_raw.groupby([aggr_level_colname]).apply(weighted_mean, environment_cols, 'AREA_HECTARES').reset_index()

    df = pd.merge(df, df_environment_aggr, on =aggr_level_colname)

    data_division           = df_cases_raw[['date','train_1','target_1', 'train_2', 'target_2','train_3', 'target_3']]
    data_division           = data_division.drop_duplicates().reset_index(drop = True)
    data_division['date']   = pd.to_datetime(data_division['date'])
    df                      = pd.merge(df, data_division, on = ['date'])

    df['cases'] = df['cases'].astype(float)
    return df

def standardize_data(df, 
                     aggr_level,
                     cheating,
                     save_data,
                     return_dfs ):
    """
    takes a merged df, and based on given parameters splits data into train and validation data, 
    adds cheating data (or not), stores the data and returns the dataframes.

    Cheating data is the data covering the gaps between train and val.
    """


    remove_columns          = ['train_1','target_1', 'train_2', 'target_2','train_3', 'target_3']
    df['population']        = df['population'].astype('float64')
    cols_to_scale           = df.select_dtypes(include=['float']).columns.tolist()
    train_dfs = []
    val_dfs   = []

    for ii in range(3):
        # index 1-3
        i       = ii + 1 
        train_i = f'train_{i}'
        val_i   = f'target_{i}'

        train_i_df = df[df[train_i] == True].reset_index(drop=True)
        val_i_df   = df[df[val_i] == True].reset_index(drop=True)

        final_train_i_date = train_i_df['date'].max()
        first_val_i_date   = val_i_df['date'].min()

        if cheating:
            cheating_data = df[(df['date'] < first_val_i_date) & (df['date'] > final_train_i_date)]
            val_i_df = pd.concat([cheating_data,val_i_df])

        # add sin / cos of week:
        cols_to_scale = cols_to_scale + ['week_sin','week_cos']

        train_i_df = add_week_features(train_i_df)
        val_i_df   = add_week_features(val_i_df)

        train = train_i_df.drop(columns = remove_columns)
        val   = val_i_df.drop(columns = remove_columns)

        train_std, std_params = pipeline_minmax_scaling(train, cols_to_scale)
        val_std               = apply_minmax_scaling(val, cols_to_scale, std_params)

        if return_dfs:
            train_dfs.append(train_std)
            val_dfs.append(val_std)

        if save_data:
            train_std.to_csv(dir_data + '/processed/' + f'train_std_{aggr_level}_{i}.csv', index=False)
            val_std.to_csv(dir_data + '/processed/' + f'val_std_{aggr_level}_{i}.csv', index=False)

            with open(dir_data + '/processed/' + f'scaling_{aggr_level}_{i}.json', 'w') as f:
                json.dump(std_params, f)   

    return train_dfs, val_dfs

def merge_standardize_data(aggr_level:str  ='state',
                           save:bool       = False,
                           cheating:bool   = False,
                           return_dfs:bool = False):
    """
    Main orchestrator function to merge and standardize data

    Parameters:
    ----------
    aggr_level: str
        aggregation level. Supported values are: "state", "maroregion" and "region"
    save: bool
        whether or not to save the final standardized and split dataframes
    cheating: bool
        whether or not to include the data within the gap between training and validation data, in the validation dataset
    return_dfs: bool
        whether or not to return a list of the final train_dfs and a list of the final val_dfs.
        If not required, put to False, as this will save memory.

    Also see:
    --------
    load_and_merge_rawdata => loads, preprocesses and merges all datasets
    standardize_data       => standardize training and validation sets separately
    """
    
    merged_data = load_and_merge_rawdata(aggr_level)

    std_data    = standardize_data(merged_data, aggr_level, save_data = save, cheating = cheating, return_dfs = return_dfs)

if __name__ == '__main__':
    for level in ['state','region','macroregion']:
        merge_standardize_data(aggr_level   = level,
                            save         = True,
                            cheating     = True,
                            return_dfs   = False)